In [1]:
import os 
os.chdir("../..")
from MSML.src.math_spec_mapping import (load_from_json, create_action_chains_graph, write_out_boundary_actions,
                      write_out_policies, write_out_mechanisms, load_svg_graphviz, write_basic_report_full,
                      write_action_chain_reports, write_spec_tree, create_parameter_impact_table,
                     write_entity_reports, write_wiring_report, write_entity_markdown_report,
                     write_all_markdown_reports)
os.chdir("Quai-Macro-Model/Spec")

from IPython.display import display, display_html
from copy import deepcopy
from src import math_spec_json
import numpy as np
from pprint import pprint
from simulation import state_base, params_base

ms = load_from_json(deepcopy(math_spec_json))

msi = ms.build_implementation(params_base)

Placeholder has no control action options
Controller Update Control Action has no control action options
No python implementation for Controller Update Policy / Linear Controller Policy. To fix this, go to Implementations/Python/Policies and add Linear Controller Policy
No python implementation for Increment Block Number Mechanism. To fix this, go to Implementations/Python/Mechanisms and add Increment Block Number Mechanism
No python implementation for Update Historical Mined Ratio Mechanism. To fix this, go to Implementations/Python/Mechanisms and add Update Historical Mined Ratio Mechanism
No python implementation for Update Historical Qi Hash Mechanism. To fix this, go to Implementations/Python/Mechanisms and add Update Historical Qi Hash Mechanism
No python implementation for Update Historical Quai Hash Mechanism. To fix this, go to Implementations/Python/Mechanisms and add Update Historical Quai Hash Mechanism
No python implementation for Log Simulation Data Mechanism. To fix this

In [5]:
state, params = msi.prepare_state_and_params(state_base, params_base, state_preperation_functions=[], parameter_preperation_functions=[])
spaces = msi.boundary_actions['Price Movements Boundary Action'](state, params, [])
pprint(spaces)
print()
print()
print()
spaces = msi.policies['Price Movements Policy'](state, params, spaces)
pprint(spaces)
print()
print()
print()
spaces = msi.mechanisms['Update Prices Mechanism'](state, params, spaces)
pprint(state)

[{'Qi Return': 0.05, 'Quai Return': 0.1}]



[{'Qi Price': 1.05, 'Quai Price': 1.1}]



{'Block Difficulty': 6000,
 'Block Number': None,
 'Dummy': None,
 'Historical Converted Qi': [],
 'Historical Converted Quai': [],
 'Historical Mined Ratio': None,
 'Historical Qi Hash': None,
 'Historical Quai Hash': None,
 'K Qi': 1,
 'K Quai': 1,
 'Metrics': {'Current Block Reward Ratio Metric': <function current_block_reward_ratio_metric at 0x7f8b684ec0d0>,
             'Hash to Qi Metric': <function hash_to_qi_metric at 0x7f8b684e0ee0>,
             'Hash to Quai Metric': <function hash_to_quai_metric at 0x7f8b684e0f70>,
             'Qi to Hash Metric': <function qi_to_hash_metric at 0x7f8b684e0dc0>,
             'Quai to Hash Metric': <function quai_to_hash_metric at 0x7f8b684e0e50>},
 'Number of Regions': 2,
 'Qi Price': 1.05,
 'Qi Supply': 10000,
 'Quai Price': 1.1,
 'Quai Supply': 10000,
 'Simulation History Log': None,
 'Stateful Metrics': {},
 'Time': None,
 'Zones per Region': 2}
